In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import JSONLoader, TextLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [7]:
# Load environment variable
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Embed the documents
#embedding models
embedding_model = "text-embedding-3-small"

#document source
doc_source_path = 'knowledgebase/'

# Chunking variables
chunk_size = 1800
chunk_overlap = 300

# Document Loader
loader = DirectoryLoader(
        doc_source_path,
        glob="**/*.txt", # "**/*.pdf" if loading pdf
        loader_cls=TextLoader,# PyMuPDFLoader if loading pdf
    )
documents = loader.load()

#Splitting documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

texts = text_splitter.split_documents(documents)

# Initialize embedding model
embeddings = OpenAIEmbeddings(model=embedding_model)

# Setting up vectorstore
vectorstore = Qdrant.from_documents(texts, embeddings,
                                    location=":memory:",
                                    collection_name="PMarca",) # "PDFCollection" if loading pdf

#Optional: Determine the length of the document loaded and vectorized
num_documents = len(texts)
print(f"Total number of documents: {num_documents}")

Total number of documents: 26


In [9]:
#Initiate document retriever 
retriever = vectorstore.as_retriever()

In [10]:
# Setting up caching system locally
set_llm_cache(SQLiteCache(database_path="cache.db"))

In [11]:
# helper function for retriever to work in LCEL chain fashion
def retrieve(inputs):
  return [doc.page_content for doc in retriever.invoke(inputs["question"])]

In [12]:
#Setting LLM parameters
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7,max_tokens=800, top_p = 1)

In [13]:
#We'll set our initial prompt - which will be initialize our LCEL chain.
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template(
    """Given {context}, answer the question `{question}` as a bullet points. Most of your response should come from the {context}.
      Be creative, concise and be as practical as possible"""
)

rag_chain = (
    RunnablePassthrough.assign(context=retrieve) | prompt | llm | StrOutputParser()
)

In [14]:
question = "what did the president say about ketanji brown?"

rag_chain.invoke({"question": question})

'- The President nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the United States Supreme Court four days ago.\n- He described her as one of the nation’s top legal minds who will continue Justice Stephen Breyer’s legacy of excellence.\n- Ketanji Brown Jackson has a background as a former top litigator in private practice and a former federal public defender.\n- She comes from a family of public school educators and police officers, emphasizing her community ties.\n- The President highlighted her ability to build consensus and noted the broad range of support she has received from various organizations, including the Fraternal Order of Police and former judges from both political parties.'